In [2]:
import os
import fitz 
# file_name = "比音勒芬.PDF"
# file_name = "天科股份2017年年度报告摘要.pdf"
# file_name = "年度报告多页.pdf"
# file_name = "深圳广田集团股份有限公司 2018 年年度报告摘要.PDF"
file_name = "20190713_国泰君安_宏观周报_ 信用流向地产和基建，制造业内外交困_花长春 韩旭 董琦 赵格格.pdf"
# file_name = "房地产行业：佛山放松人才购房政策，龙头房企11月销售成色不减－周报-东北证券191202.pdf"
full_path = os.getcwd()+"/pdfs/pdf_catas/"+file_name

doc = fitz.open(full_path)

#### 将fitz生成的多个block进行合并，形成多个region
- 图片
- 表格
- 内容
- 页眉/页脚

In [12]:
class Block(object):
    def __init__(self,left_down,right_up, text):
        """left_down = [],right_up = []"""
        self.left_down = left_down
        self.right_up = right_up
        self.text = text

    def get_center(self):
        """"计算block的中心坐标"""
        return [(self.left_down[0]+self.right_up[0])/2,(self.left_down[1]+self.right_up[1])/2]

    def get_width(self):
        return abs(self.left_down[0]-self.right_up[0])
    
    def get_height(self):
        return abs(self.left_down[1]-self.right_up[1])

In [13]:
"""
矩形的坐标（左下角，右上角）
设置 x_Threshold,y_Threshold
"""
class Region(Block):
    """多个block聚合形成的"""
    def __init__(self,left_down,right_up):
        """left_down = [],right_up = []"""
        self.left_down = left_down
        self.right_up = right_up
        self.texts = []

    def cal_dist(self,block_center):
        cluster_center = self.get_center()
        x_d = abs(cluster_center[0] - block_center[0])
        y_d = abs(cluster_center[1] - block_center[1])
        return [x_d,y_d]

    def add_block(self,block):
        """region和block组合新的region"""
        self.left_down = [min(block.left_down[0],self.left_down[0]),min(block.left_down[1],self.left_down[1])]
        self.right_up = [max(block.right_up[0],self.right_up[0]),max(block.right_up[1],self.right_up[1])]
        self.texts.append(block.text)

In [3]:
page0 = doc[0] # 选择page
fitz_blocks = fitz.utils.getTextBlocks(page0, images=True)
fitz_blocks

[(86.90399932861328,
  803.2255859375,
  231.4759979248047,
  812.2255859375,
  '请务必阅读正文之后的免责条款部分',
  0,
  0),
 (56.7599983215332,
  76.27198791503906,
  110.75999450683594,
  92.8919906616211,
  '2019.07.13',
  1,
  0),
 (85.8239974975586,
  106.78437805175781,
  358.720703125,
  122.74437713623047,
  '信用流向地产和基建，制造业内外交困',
  2,
  0),
 (125.54000091552734,
  133.43585205078125,
  388.0682067871094,
  152.1652069091797,
  '——6 月金融、贸易、通胀数据和宏观周报',
  3,
  0),
 (51.119998931884766,
  160.5003204345703,
  90.68832397460938,
  174.5873565673828,
  '[Table_A',
  4,
  0),
 (51.099998474121094,
  166.1999969482422,
  63.599998474121094,
  177.5,
  '<image: DeviceRGB, width 21, height 19, bpc 8>',
  5,
  1),
 (91.34400177001953,
  165.583984375,
  386.8269958496094,
  178.04898071289062,
  '花长春(分析师) 韩旭(研究助理) 董琦(研究助理) 赵格格(研究助理)',
  6,
  0),
 (51.099998474121094,
  180.5,
  63.849998474121094,
  191.75,
  '<image: DeviceRGB, width 21, height 19, bpc 8>',
  7,
  1),
 (91.34400177001953,
  180.8192901

In [226]:
page0

page 5 of /home/ginger/Projects/Learning/P_Stack/file_tools/pdf_tools/pdfs/pdf_catas/20190713_国泰君安_宏观周报_ 信用流向地产和基建，制造业内外交困_花长春 韩旭 董琦 赵格格.pdf

In [227]:
# fitz_blocks

In [228]:
blocks = fitz_blocks
# blocks[0]

In [229]:
block_objs = []

for block in blocks:
    block_objs.append(Block([block[0],block[1]],[block[2],block[3]],block[4]))
blocks = block_objs

In [230]:
# def draw_rect_on_page(blocks):


In [231]:
def metric_block_with_regions(block,regions):
    """度量block和regions
    如果block和region满足关系则将block加入该region，
    否则，block成为新的regions，并加入新的regions
    """
    nearest = cal_metric(regions=regions,block=block)
    # print(nearest)
    if nearest:
        regions[nearest].add_block(block)
    else:
        # return regions
    # 当前block不加入cluster
        region = Region(block.left_down,block.right_up)
        region.add_block(block)
        regions.append(region)
    return regions

In [232]:
def cal_metric(regions,block):
    block_center = block.get_center() # 计算block的中点
    block_width = block.get_width()
    block_height = block.get_height()
    nearest = None
    cur_h_dist = 10000
    cur_v_dist = 10000
    for i in range(len(regions)):
        region = regions[i]
        center_dist = region.cal_dist(block_center)
        horizontal_dist = center_dist[0]  # 水平距离 边框之间的距离
        vertical_dist = center_dist[1] - (region.get_height()/2+block_height/2)  # 垂直距离
        # print(horizontal_dist,vertical_dist)
        if horizontal_dist < max(region.get_width()/2,block_width/2) and (vertical_dist < 20):
        # if True:
            if  horizontal_dist <= cur_h_dist and vertical_dist <= cur_v_dist:
                cur_h_dist = horizontal_dist
                cur_v_dist = vertical_dist
                nearest = i
    return nearest

In [233]:
d = {9: [10, 6], 8: [9, 7]}
sorted(d.items(), key=lambda x :(x[1]))

[(8, [9, 7]), (9, [10, 6])]

In [234]:
xs = [[10, 6],[9, 7]]
sorted(xs, key=lambda x :(x[0], x[1]))

[[9, 7], [10, 6]]

In [235]:
# def find_nearest_block(metrics):
#     """
#     寻找最近邻的block
#     """
#     for m in metrics:


In [236]:
blocks = sorted(blocks, key=lambda x: (x.left_down[1],x.left_down[0]))
for b in blocks:
    print(b.left_down,b.right_up)

[100.0, 9.88873291015625] [331.0, 23.88873291015625]
[51.04999923706055, 34.75] [210.8000030517578, 58.75]
[455.5, 48.683956146240234] [544.2999877929688, 61.148956298828125]
[51.0, 78.75532531738281] [444.3138427734375, 93.38092803955078]
[321.1070251464844, 100.25543212890625] [325.1996154785156, 105.85711669921875]
[58.05487823486328, 103.08866119384766] [81.28107452392578, 113.35000610351562]
[311.15374755859375, 107.40115356445312] [316.3518981933594, 112.55984497070312]
[311.15374755859375, 122.85151672363281] [316.3518981933594, 128.01022338867188]
[58.05487823486328, 130.5665283203125] [63.75887680053711, 136.2176971435547]
[311.15374755859375, 138.29617309570312] [316.3518981933594, 143.45486450195312]
[58.05487823486328, 153.4530029296875] [63.75887680053711, 159.1041717529297]
[311.15374755859375, 153.74655151367188] [316.3518981933594, 158.90524291992188]
[311.15374755859375, 169.19119262695312] [316.3518981933594, 174.34988403320312]
[58.05487823486328, 176.3394775390625] 

In [237]:
Regions = []

for block in blocks:
    metric_block_with_regions(block,Regions)
    # print(Regions)

# print(Regions[0].texts)
print(len(Regions))
for c in Regions:
    # if len(c.texts)>1: 
    print(c.texts)

13
['国泰君安版权所有发送给：李卓轩 p6']
['<image: Indexed(255,DeviceRGB), width 213, height 32, bpc 8>']
['日报/周报/双周报/月报']
['图 4：7 月高炉开工率小幅下跌 图 5： 7 月日均耗煤同比下降', '%', '76\n%', '50', '40', '72', '30', '68', '20', '10', '64', '0', '60', '-10', '-20', '2016-07 2017-01 2017-07 2018-01 2018-07 2019-01 2019-07', '高炉开工率:全国', '日均耗煤量:6大发电集团:合计:月:同比', '资料来源：WIND，国泰君安证券研究', '图 6：7 月水泥价格下跌，环比增速略有下降 图 7：7 月动力煤价格上涨，同比增速略有回升', '%\n元/吨', '%', '180', '30', '800', '750', '20', '160', '700', '10', '650', '140', '600', '0', '550', '120', '-10', '500', '100', '450', '-20', '400', '80', '-30', '350', '300', '60', '-40', '2016-07 2017-01 2017-07 2018-01 2018-07 2019-01 2019-07', '秦皇岛港:平仓价:动力末煤(Q5500):山西产:月\n秦皇岛港:平仓价:动力末煤(Q5500):山西产:月:环比\n秦皇岛港:平仓价:动力末煤(Q5500):山西产:月:同比', '水泥价格指数:全国:月 水泥价格指数:全国:月:环比 水泥价格指数:全国:月:同比', '资料来源：WIND，国泰君安证券研究']
['120']
['90']
['60']
['30']
['0']
['-30', '2016-07 2017-01 2017-07 2018-01 2018-07 2019-01 2019-07']
['2.3.上游：原油价格小幅上升', '7 月以来，原油价格小幅上升。截至 7 月 12 日，英国布伦特原油现货价格为 67.46\n美元/桶，较 6 月环比上升 0.5%；WT

In [184]:
coordinates_x = []
coordinates_y = []
for b in blocks[1:20]:
    coordinates_x.append(b[0])
    coordinates_x.append(b[2])
    coordinates_y.append(b[1])
    coordinates_y.append(b[3])

"""
左下角 右上角
(86.90399932861328,
  803.2255859375,
  231.4759979248047,
  812.2255859375)
"""
import matplotlib.pyplot as plt
plt.scatter(coordinates_x, coordinates_y)

TypeError: 'Block' object does not support indexing

In [224]:
def is_head_or_foot(page_height, char):
    """
    判断是否是页眉页脚
    :param page_height: 页面高度
    :param char: 需要判断的字符
    :return: True or False
    """
    head = page_height - 50
    foot = 50
    x0, y0, x1, y1 = char['x0'], char['bottom'], char['x1'], char['top']
    if head < y1 or y0 < foot:
        return True